# Astra Zeneca Covid Vaccine - EMA risk analysis
> A tutorial of fastpages for Jupyter notebooks.

- toc: true 
- badges: true
- comments: false
- categories: [jupyter]
- image: images/chart-preview.png

In [22]:
#hide
import pandas as pd
from fastdata.integrations import *
from fastdata.core import *
import plotly.express as px
from IPython.display import HTML

## Goal

The goal of this analysis is to help you understand what the deal is with the Astra Zeneca vaccine, specifically the risks reported by the media and the conclusions of the last European Medical Agency (EMA) report.

The report is based on EMA data published [here](https://www.ema.europa.eu/en/documents/prac-recommendation/signal-assessment-report-embolic-thrombotic-events-smq-covid-19-vaccine-chadox1-s-recombinant-covid_en.pdf). 

The analysis does not aim to challenge the underlying data, but rather make part of the report easier to understand without going through a 50 page EMA report.

> Warning: DISCLAIMER: The author is not an expert in the field, and is applying some general statistical thinking to the problem. Therefore, it may contain errors, omissions or otherwise not accurate information.

## Methodology

### Introduction

The EMA report performs some observed to expected analysis (OE) in the report to try to understand the potential risks of the vaccine. We will focus on the "EMA analysis of EudraVigilance data" (sction 3.1.5 in the report).

**Expected to observed analysis:**

The logic of this analysis is to compare how many cases you have observed with one condition (observed) vs. how many usually happen (expected). With this, you can calculate an Observed to Expected Ration, which is defined as # of observed cases / # expected cases.

The statistical uncertainty will often be driven by the observed number of cases, which is often small (rare events). To deal with this statistical uncertainty around the total number of cases observed over the risk period of interest, a 95% Poisson exact confidence interval (95%CI) is often used (more on this later).

**EudraVigilance database:**

EudraVigilance is a database with information about suspected adverse reactions to medicines which have been authorised or being studied in clinical trials in the European Economic Area (EEA). 
[Source](https://www.ema.europa.eu/en/human-regulatory/research-development/pharmacovigilance/eudravigilance)

This section performs an OE analysis 3 types of conditions present in the EudraVigilance database:
- Disseminated intravascular coagulation
- Cerebral Venous Sinus Thrombosis
- Embolic and thrombotic events

### Data sources

A key input for the analysis is the incidence rate of the specific condition, to be able to determine the expected cases. It is useful also if the data is stratified by groups, to be able to analyize not just the general population as a whole but also individual subgroups.

The databases used for the main analysis for the three events investigated are:
- Coagulation disorder (this was used to compare with the SMQ Embolic and
thrombotic events): ARS from Italy
- Disseminated intravascular coagulation: FISABIO from Spain
- Cerebral venous sinus thrombosis: ARS from Italy 

## Analysis of potential side-effects

### Disseminated intravascular coagulation (DIC)

Disseminated intravascular coagulation (DIC) is a rare but serious condition that causes abnormal blood clotting throughout the body’s blood vessels. It is caused by another disease or condition, such as an infection or injury, that makes the body’s normal blood clotting process become overactive.

[Source: US NIH](https://www.nhlbi.nih.gov/health-topics/disseminated-intravascular-coagulation)

For those of us that are not medicine exprts, this diagram shows of a thrombus (blood clot) that has blocked a blood vessel valve.

![](https://upload.wikimedia.org/wikipedia/commons/c/c5/Blood_clot_diagram.png)

In [23]:
#hide
dic = gsheet_to_df(
    url="https://docs.google.com/spreadsheets/d/11yJ8GbArmcazWG8UdsSWD2gY2VIVD7l_zaPOdiF2ePY", 
    start_row=2, 
    sheet="DIC")

In [24]:
#hide
dic = dic.drop(
    columns=["EEA Expected 14d","EEA Observed 14d From EV","EEA OE 14d with 95% c.i."])

In [25]:
#hide
dic["oe_ci_interval_min"] = dic["EEA+UK  OE 14d with 95% c.i."].fdt.clean_text_column(
    mode="custom", 
    keep_unmatched=False, 
    regex="(\d+?[,.]\d+) - \d+?[,.]\d+")

In [26]:
#hide
dic["oe_ci_interval_max"] = dic["EEA+UK  OE 14d with 95% c.i."].fdt.clean_text_column(
    mode="custom", 
    keep_unmatched=False, 
    regex="\d+?[,.]\d+ - (\d+?[,.]\d+)")

In [27]:
#hide
dic["oe"] = dic["EEA+UK  OE 14d with 95% c.i."].fdt.clean_text_column(
    mode="before_character", 
    keep_unmatched=False, 
    character="(")

In [28]:
#hide
dic = dic.drop(
    columns=["EEA+UK  OE 14d with 95% c.i."])

In [29]:
#hide
dic = dic.astype(
    dtype={"IR per 100,000 Person years From FISABIO" : "float64", "EEA+UK Expected 14d" : "float64", "EEA+UK  Observed 14d From EV" : "float64", "oe_ci_interval_min" : "float64", "oe_ci_interval_max" : "float64", "oe" : "float64"})

#### Expected vs. observed analysis

We see that for age groups below 50, there are more expected cases than observed cases, **and therefore it is clear there is no risk**.

In [30]:
#hide_input
HTML(px.bar(dic,
    title="Expected vs. observed cases", 
    barmode="group", 
    template="seaborn", 
    x=["EEA+UK Expected 14d","EEA+UK  Observed 14d From EV"], 
    y="Age group").to_html(include_plotlyjs='cdn'))

In [31]:
#hide
dic = dic.query("(`Age group`=='20-29' or `Age group`=='30-49')", engine="python").copy()

In [32]:
#hide
dic["error_above_oe"] = dic["oe_ci_interval_max"].subtract(
    other=dic["oe"])

In [33]:
#hide
dic["error_below_oe"] = dic["oe"].subtract(
    other=dic["oe_ci_interval_min"])

But for ages below 50, there are more observed cases than expected.

Let's run through one example to understand this better:
- In the case of 30-49, we expect 2 cases but get 4. This means the ratio of observed to expected is about 2.
- But the confidence interval tells us that in 95% of cases, the number of observed will fall between 0.54 and 5.16. 
- This means that if cases appear with a certain probability, there are chances where you will get less and chances you will get more, but you 95% of the time, you will get between 0.54 and 5.16. You can think of flipping a coin, where on average, you get 50% heads or tails, but within a ten coin flip, you could get more heads or more tails.
- This means that be able to conclude statistically that there is something unusual that cloud not likely be attributed to chance, you need a number above 5.16, which is not the case (2<5.16)
- Intuitively, you can understand this as the numbers are so small, one small change could very easily skew the result, and thus it is not easy to know for sure with so few cases

**The conclusion is that for these groups (of 30-49 and especially for 20-29) the fact that the observed is greater than the expected is not statistically significant.**

In [34]:
#hide_input
HTML(px.scatter(dic,
    title="Observed to expected ration vs. confidence interval",
    y="Age group", 
    x="oe", 
    labels={"oe":"observed/expected"},
    template="seaborn", 
    error_x="error_above_oe", 
    error_x_minus="error_below_oe").to_html(include_plotlyjs='cdn'))

Something worth discussing is that many of the patients who are getting the vaccinations are probably not the most healthy (if we assume that some rational prioritization is taking place). Given this, it is fair to assume that diseases' incidence may be higher in this group than in the general population.

Unfortunately, it seems that with the given data, we can't control for that.
And this brings us to one of the main conclusion of this analysis: **We don't have good health data to answer these questions.**. For example, it is not easy to get the incidence rate breakdown by male/female or by pre-conditions and compare it with the observed cases.

### Embolic and thrombotic events

In [35]:
#hide
et = gsheet_to_df(
    url="https://docs.google.com/spreadsheets/d/11yJ8GbArmcazWG8UdsSWD2gY2VIVD7l_zaPOdiF2ePY", 
    start_row=2, 
    sheet="ET")

In [36]:
#hide
et["oe_ci_interval_min"] = et["EEA OE 14d with 95% c.i."].fdt.clean_text_column(
    mode="custom", 
    keep_unmatched=False, 
    regex="(\d+?[,.]\d+) - \d+?[,.]\d+")

In [37]:
#hide
et["oe_ci_interval_max"] = et["EEA OE 14d with 95% c.i."].fdt.clean_text_column(
    mode="custom", 
    keep_unmatched=False, 
    regex="\d+?[,.]\d+ - (\d+?[,.]\d+)")

In [38]:
#hide
et["oe"] = et["EEA OE 14d with 95% c.i."].fdt.clean_text_column(
    mode="before_character", 
    keep_unmatched=False, 
    character="(")

In [39]:
#hide
et = et.drop(
    columns=["EEA OE 14d with 95% c.i."])

In [40]:
#hide
et = et.astype(
    dtype={
           "EEA Expected 14d" : "float64", 
           "EEA Observed 14d From EV" : "float64", 
           "oe_ci_interval_min" : "float64", 
           "oe_ci_interval_max" : "float64", 
           "oe" : "float64"})

#### Expected vs. observed analysis

In this case, all expected cases are above the observed except for the group of 30-49

In [41]:
#hide_input
HTML(px.bar(et,
    title="Expected vs. observed cases", 
    barmode="group", 
    template="seaborn", 
    x=["EEA Expected 14d","EEA Observed 14d From EV"], 
    y="Age group").to_html(include_plotlyjs='cdn'))

In [42]:
#hide
et = dic.query("(`Age group`=='30-49')", engine="python").copy()

In [43]:
#hide
et["error_above_oe"] = et["oe_ci_interval_max"].subtract(
    other=et["oe"])

In [44]:
#hide
et["error_below_oe"] = et["oe"].subtract(
    other=et["oe_ci_interval_min"])

As before, we see that the value of observed to expected falls within the 95% interval.

In [45]:
#hide_input
HTML(px.scatter(et,
    title="Observed to expected ration vs. confidence interval",
    y="Age group", 
    x="oe", 
    labels={"oe":"observed/expected"},
    template="seaborn", 
    error_x="error_above_oe", 
    error_x_minus="error_below_oe").to_html(include_plotlyjs='cdn'))

### Cerebral Venous Sinus Thrombosis

#### Load and clean the data

In [46]:
#hide
cvst = gsheet_to_df(
    url="https://docs.google.com/spreadsheets/d/11yJ8GbArmcazWG8UdsSWD2gY2VIVD7l_zaPOdiF2ePY", 
    start_row=2, 
    sheet="CVST")

In [47]:
#hide
cvst = cvst.drop(
    columns=["EEA Expected 14d","EEA Observed 14d From EV","EEA OE 14d with 95% c.i."])

In [48]:
#hide
cvst["oe_ci_interval_min"] = cvst["EEA+UK OE 14d with 95% c.i."].fdt.clean_text_column(
    mode="custom", 
    keep_unmatched=False, 
    regex="(\d+?[,.]\d+) - \d+?[,.]\d+")

In [49]:
#hide
cvst["oe_ci_interval_max"] = cvst["EEA+UK OE 14d with 95% c.i."].fdt.clean_text_column(
    mode="custom", 
    keep_unmatched=False, 
    regex="\d+?[,.]\d+ - (\d+?[,.]\d+)")

In [50]:
#hide
cvst["oe"] = cvst["EEA+UK OE 14d with 95% c.i."].fdt.clean_text_column(
    mode="before_character", 
    keep_unmatched=False, 
    character="(")

In [51]:
#hide
cvst = cvst.drop(
    columns=["EEA+UK OE 14d with 95% c.i."])

In [52]:
#hide
cvst = cvst.astype(
    dtype={"IR per 100,000 Person years From ARS" : "float64", 
           "EEA+UK Expected 14d" : "float64", 
           "EEA+UK Observed 14d From EV" : "float64", 
           "oe_ci_interval_min" : "float64", 
           "oe_ci_interval_max" : "float64", 
           "oe" : "float64"})

#### Expected vs. observed analysis

We see that for age groups below 50, there are more expected cases than observed cases, **and therefore it is clear there is no risk**.

In [53]:
#hide_input
HTML(px.bar(cvst,
    title="Expected vs. observed cases", 
    barmode="group", 
    template="seaborn", 
    x=["EEA+UK Expected 14d","EEA+UK Observed 14d From EV"], 
    y="Age group").to_html(include_plotlyjs='cdn'))

In [54]:
#hide
cvst = cvst.query("(`Age group`=='20-29' or `Age group`=='30-49')", engine="python").copy()

In [55]:
#hide
cvst["error_above_oe"] = cvst["oe_ci_interval_max"].subtract(
    other=cvst["oe"])

In [56]:
#hide
cvst["error_below_oe"] = cvst["oe"].subtract(
    other=cvst["oe_ci_interval_min"])

In [57]:
#hide_input
HTML(px.scatter(cvst,
    title="Observed to expected ration vs. confidence interval",
    y="Age group", 
    x="oe", 
    labels={"oe":"observed/expected"},
    template="seaborn", 
    error_x="error_above_oe", 
    error_x_minus="error_below_oe").to_html(include_plotlyjs='cdn'))